In [2]:
import pandas as pd
import numpy as np

In [3]:
import auth
import getpass

user  = getpass.getuser()
token = auth.get_valid_token()

In [4]:
import trino
import os
connection = trino.dbapi.connect(
    host=os.environ['PRESTO_HOST'],
    port=443,
    user=user,
    catalog="hive",
    schema="dwh_insider",
    http_scheme='https',
    auth=trino.auth.BasicAuthentication(user, token))

### Claims Data Table 

In [5]:
# Taking the claim information from the datalake
query = """
WITH claim_query as 
(
SELECT

       carrier_tracking_number
      , claim_shipment_number
      , sk_claim_report_date
      , aviailable_claim_amount_euro
      , net_claim_amount_euro
      , carrier_name
      , carrier_name_short
      , carrier_country_code
      , claimtype
      , cast(sk_month as int) as sk_month
      , SK_WEEK
      , claimsubtype
      , manual_refund_reason_name
      , claim_category
      , dd.ZIP
      , dd.city
      , TS_LAST_COT_EVENT
      , TS_LAST_CRT_EVENT
      , unplanned_return_flag 
      , reconditioned_claim_flag 
      , CASE WHEN chan.sk_cfa in (31, 26) THEN 'FashionStore'
             WHEN chan.sk_cfa = 28 THEN 'Lounge' END AS cfa -- 26 is empty for recent periods, it stands for zlash
FROM
    dwh_confidential.f_clm_claim_shipments as fccs
LEFT JOIN
          dwh_insider.d_clm_carrier_liability as dccl
          ON dccl.sk_clm_carrier_liability = fccs.sk_clm_carrier_liability
LEFT JOIN
          dwh_insider.d_network_carriers as dnc
          ON dnc.sk_network_carrier = dccl.sk_network_carrier
LEFT JOIN
          dwh_insider.d_sf_carrier_claim_types as dscct
          ON dscct.sk_claim_type = fccs.sk_claim_type
LEFT JOIN
          dwh_insider.d_clm_claim_categories as dccc
          ON dccc.sk_claim_category = fccs.sk_claim_category
LEFT JOIN
          dwh_insider.d_manual_refund_reason AS dmrr
          ON dmrr.sk_manual_refund_reason = fccs.sk_manual_refund_reason
LEFT JOIN
          dwh_confidential.D_DESTINATIONS AS dd
          ON fccs.SK_DESTINATION = dd.SK_DESTINATION
LEFT JOIN
          dwh_insider.d_sales_channels AS chan
          ON fccs.sk_sales_channel = chan.sk_sales_channels 
LEFT JOIN 
          dwh_insider.d_date as d
          ON d.sk_date=fccs.sk_claim_report_date
          
WHERE  chan.sk_cfa in (31, 26, 28)
)
SELECT COUNT(DISTINCT claim_shipment_number) as number_of_claims,
       SUM(aviailable_claim_amount_euro) as claims_amount ,
       sk_month
FROM claim_query
 WHERE 
 
      sk_month Between 202201 AND 202212
GROUP BY sk_month
   """
df = pd.read_sql_query(query, connection)
df    

/tmp/ipykernel_341/1171088982.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,number_of_claims,claims_amount,sk_month
0,42922,4468807.1183000,202205
1,33928,2789338.5664000,202208
2,37956,3609728.2585000,202210
3,58156,4622981.2526000,202212
4,39101,4857672.8195000,202203
5,43827,3505862.3822000,202207
6,54550,4727421.6744500,202211
7,32880,3143043.7779000,202209
8,38397,4230946.1040000,202204
9,51726,5411668.6775000,202201
